In [183]:
import requests
import time
import dateparser
import pytz
from datetime import datetime
from pandas import DataFrame as df

# python-binance的package
from binance.client import Client

import pandas as pd

In [184]:
# client = Client('lZSTGrwn4WuX19XHkUu1PSejTZvBVJW6Z9Uq0TVjPWaDrW3DXSEg1OYwnZYKf4P8', 'dSCdt3keoNPWylWb67KzZFzIhknMPBl3QqBNw8RPMxgTeODtwtjaRTZPfkymwwgk')

In [185]:
def date_to_milliseconds(date_str):
    """Convert UTC date to milliseconds
    If using offset strings add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"
    See dateparse docs for formats http://dateparser.readthedocs.io/en/latest/
    :param date_str: date in readable format, i.e. "January 01, 2018", "11 hours ago UTC", "now UTC"
    :type date_str: str
    """
    # get epoch value in UTC
    epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
    # parse our date string
    d = dateparser.parse(date_str)
    # if the date is not timezone aware apply UTC timezone
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        d = d.replace(tzinfo=pytz.utc)

    # return the difference in time
    return int((d - epoch).total_seconds() * 1000.0)

def interval_to_milliseconds(interval):
    """Convert a Binance interval string to milliseconds
    :param interval: Binance interval string 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w
    :type interval: str
    :return:
         None if unit not one of m, h, d or w
         None if string not in correct format
         int value of interval in milliseconds
    """
    ms = None
    seconds_per_unit = {
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60
    }

    unit = interval[-1]
    if unit in seconds_per_unit:
        try:
            ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
        except ValueError:
            pass
    return ms

In [186]:
def get_fundingRate(symbol, start_str, end_str):
    start_ts = date_to_milliseconds(start_str)
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)
        end_ts += interval_to_milliseconds('8h')
    limit = 1000
    
    params = {
        'symbol': symbol,
        'startTime': start_ts,
        'endTime': end_ts,
        'limit':limit
    }
    response = requests.get("https://fapi.binance.com/fapi/v1/fundingRate", params)
    data = response.json()
    
    # ret = []
    ret = []
    for i in range(len(data)):
        ret.append(data[i]['fundingRate'])
    return ret
# get_fundingRate("BTCUSDT","1 Fab, 2021","5 Fab, 2021")

def get_klines(symbol, start_str, end_str, interval):
    client = Client("", "")
    
    start_ts = date_to_milliseconds(start_str)
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)
    inter = interval_to_milliseconds('8h')
    if interval:
        inter = interval_to_milliseconds(interval)
    
    limit = 1000
    data = []
    
    while (end_ts-start_ts)/inter+1 > limit:
        k = client.get_klines(symbol=symbol,
                             interval=interval,
                             limit=limit,
                             startTime=start_ts)
        data = data+k
        start_ts += (limit+1)*inter
    k = client.get_klines(symbol=symbol,
                             interval=interval,
                             limit=limit,
                             startTime=start_ts,
                             endTime=end_ts)
    
    data = data+k
    return(data)

In [187]:
# klines = get_klines("BTCUSDT","1 Fab, 2021","5 Fab, 2021", '2h')
#時間
def get_time(klines):
    ret = []
    for i in range(len(klines)): 
        # print(datetime.fromtimestamp(round(int(klines[i][0]/1000),3)))
        readable_time = datetime.fromtimestamp(round(int(klines[i][0]/1000),3))
        ret.append(readable_time)
        dataframe_final_date = df(ret)
    return dataframe_final_date
# 收盤價
def get_close(klines):
    ret = []
    for i in range(len(klines)):
        ret.append(round(float(klines[i][4]),5))
    return ret
# get_close(klines)

#波動（max-min)/開盤
def get_fluctuation(klines):
    ret = []
    for i in range(len(klines)):
        temp = round((float(klines[i][2])-float(klines[i][3]))/float(klines[i][1]),3)
        ret.append(temp)
    # print(len(ret) == len(klines))
    return ret
# get_fluctuation(klines)

# （收盤-開盤）/開盤
def get_change_rate(klines):
    ret = []
    for i in range(len(klines)):
        temp = round((float(klines[i][4])-float(klines[i][1]))/float(klines[i][1]),3)
        ret.append(temp)
    # print(len(ret) == len(klines))
    return ret
# get_change_rate(klines)

# 成交訂單數
def get_dealNumbers(klines):
    ret = []
    for i in range(len(klines)):
        ret.append(klines[i][8])
    return ret     
# get_dealNumbers(klines)

#成交額
def get_turnover(klines):
    ret = []
    for i in range(len(klines)):
        ret.append(float(klines[i][7]))
    return ret

#成交量
def get_volume(klines):
    ret = []
    for i in range(len(klines)):
        ret.append(round(float(klines[i][5]),3))
    return ret
# get_volume(klines)

#主動成交量除以成交量
def get_proactiveRate(klines):
    ret = []
    for i in range(len(klines)):
        ret.append(round(float(klines[i][9])/float(klines[i][5]), 3))
    return ret
# get_proactiveRate(klines)

In [188]:
#當前未平倉合約->指回傳一個數據
def get_openInterest(symbol):  
    params = {
        'symbol': symbol,
    }
    response = requests.get("https://fapi.binance.com/fapi/v1/openInterest", params)
    data = response.json()
    return data['openInterest']
# get_openInterest("BTCUSDT")

#這個只有30天內的數據
def get_sumOpenInterestValue(symbol, period, start_str, end_str):
    start_ts = date_to_milliseconds(start_str)
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)
    limit = 500
        
    params = {
        'symbol': symbol,
        'period': period,
        'startTime': start_ts
#         'endTime': end_ts,
#         'limit':limit
    }
    response = requests.get("https://fapi.binance.com/futures/data/openInterestHist", params)
    data = response.json()
    ret = []
    for i in range(len(data)):
        ret.append(round(float(data[i]['sumOpenInterestValue']),3))
    return ret

# get_sumOpenInterestValue('BTCUSDT', '12h', "1 Jun, 2021","5 Jun, 2021")

In [189]:
#這兩個只有30天內的數據

def get_topLongShortAccountRatio(symbol, period, start_str = None, end_str = None):
    start_ts = None
    if start_ts:
        start_ts = date_to_milliseconds(start_str)
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)
    limit = 500
        
    params = {
        'symbol': symbol,
        'period': period,
        'startTime': start_ts,
        'endTime': end_ts,
        'limit':limit
    }
    response = requests.get("https://fapi.binance.com/futures/data/topLongShortAccountRatio", params)
    data = response.json()
    ret = []
    for i in range(len(data)):
        ret.append(round(float(data[i]["longShortRatio"]),3))
    return ret
# get_topLongShortAccountRatio('BTCUSDT', '2h', "3 Jun, 2021","5 Jun, 2021")

def get_globalLongShortAccountRatio(symbol, period, start_str = None, end_str = None):
    start_ts = None
    if start_ts:
        start_ts = date_to_milliseconds(start_str)
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)
    limit = 500
        
    params = {
        'symbol': symbol,
        'period': period,
        'startTime': start_ts,
        'endTime': end_ts,
        'limit':limit
    }
    response = requests.get("https://fapi.binance.com/futures/data/globalLongShortAccountRatio", params)
    data = response.json()
    ret = []
    for i in range(len(data)):
        ret.append(round(float(data[i]["longShortRatio"]),3))
    return ret
# get_globalLongShortAccountRatio('BTCUSDT', '12h', "3 Jun, 2021","5 Jun, 2021")

In [190]:
def write_close_toCsv(path):
    data = pd.DataFrame()
    length = 0
    for x in symb:
        if check_exist(x, start, interval):
            symbol = x
            klines = []
            klines = get_klines(x, start, end, interval)
            time = get_time(klines)
            close = get_close(klines)
            if x=='BTCUSDT':
                length = len(klines)
    #             print(length)
            if(len(klines)==length):
                data[x] = close
    data.insert(0, "Date",time, True)
    data.to_csv(path, encoding = 'utf-8', index = False)
    
def get_close_DataFrame():
    data = pd.DataFrame()
    length = 0
    for x in symb:
        if check_exist(x, start, interval):
            symbol = x
            klines = []
            klines = get_klines(x, start, end, interval)
            time = get_time(klines)
            close = get_close(klines)
            if x=='BTCUSDT':
                length = len(klines)
    #             print(length)
            if(len(klines)==length):
                data[x] = close
    data.insert(0, "Date",time, True)
    return data

In [191]:
def get_changeRate_DataFrame():
    data = pd.DataFrame()
    length = 0
    for x in symb:
        symbol = x
        klines = []
        klines = get_klines(x, start, end, interval)
        # print(len(klines[0]))
        ch_rate = get_change_rate(klines)
        if x=='BTCUSDT':
            length = len(klines)
        if(len(klines)==length):
            data[x] = ch_rate
    return data

def check_exist(symbol, start, interval):
    klines = []
    klines = get_klines(symbol, start, end, interval)
    # print(len(klines[0]))
    return klines != []

In [192]:
def write_symbol_dataFrame(symbol, start, end, interval = '8h'):
    print(symbol, start, end)
    data = pd.DataFrame()
    if check_exist(symbol, start, interval):
        klines = []

        klines = get_klines(symbol, start, end, interval)
        data['close'] = get_close(klines)
        time = get_time(klines)
        # data['volume'] = get_volume(klines)
        # data['turnover'] = get_turnover(klines)
        # data['change_rate'] = get_change_rate(klines)
        # data['dealNumbers'] = get_dealNumbers(klines)
        # data['flu０tuation'] = get_fluctuation(klines)
        # data['proactiveRate'] = get_proactiveRate(klines)
        # data['FundingRate'] = get_fundingRate(symbol,start, end)
    else:
        print("not exist")
    data.insert(0, "Date",time, True)
    return data



In [193]:
# 全部的function

# klines = []
# klines = get_klines("BTCUSDT","1 Fab, 2021","5 Fab, 2021", '12h')
# get_proactiveRate(klines)
# get_volume(klines)
# get_turnover(klines)
# get_change_rate(klines)
# get_dealNumbers(klines)
# get_fluctuation(klines)
# get_close(klines)

# get_globalLongShortAccountRatio('BTCUSDT', '12h', "3 Jun, 2021","5 Jun, 2021")
# get_topLongShortAccountRatio('BTCUSDT', '2h', "3 Jun, 2021","5 Jun, 2021")
# get_sumOpenInterestValue('BTCUSDT', '12h', "1 Jun, 2021","5 Jun, 2021")
# get_openInterest("BTCUSDT")

# get_fundingRate("BTCUSDT","1 Fab, 2021","5 Fab, 2021")

In [194]:
# sym = ["BTC", "ETH", "BNB", "ADA", "DOGE", "XRP", "DOT", "BCH", "UNI", "LTC", "SOL", "THETA", "LINK", "XLM", "MATIC",
# "VET", "ETC", "FIL", "DAI", "ALGO", "xmr", "trx", "eos", "shib", "ftt", "neo", "aave",
# "atom", "cake", "ksm", "iota", "luna", "xtz", "mkr", "tfuel", "btt",
# "waves", "dcr", "zec", "chz", "rune", "dash", "avax", "egld", "xem", "hot", "hbar", "enj", "hnt", "snx",
# "comp", "zil", "yfi", "mdx", "sushi", "btg", "bat", "zen", "mana", "qtum", "nano", "stx", "ar",
# "grt", "dgb", "ont", "omg", "crv", "icx", "sc", "rvn", "uma", "zrx", "ong", "ftm", "lpt", "near",
# "1inch", "ckb", "xvg", "one", "iost", "lsk", "ren", "win", "knc", "lrc", "celo",
# "rlc", "kava", "rsr", "ogn", "wrx", "vtho", "dent", "bnt", "nmr", "keep", "iotx", "cfx", "ankr", "srm", "sxp", 
# "tko", "axs", "dodo"]

sym = ["BTC", "WRX"]

symb=[]
for x in sym:
    symb.append(str.upper(x)+"USDT")
# 這幾個資料今年二月份還沒有
# DAIUSDT
# SHIBUSDT
# CAKEUSDT
# MDXUSDT
# BTGUSDT
# ARUSDT
# LPTUSDT
# XVGUSDT
# KEEPUSDT
# CFXUSDT
# TKOUSDT
# DODOUSDT
# 這幾個資料今年三月份還沒有
# DAIUSDT
# SHIBUSDT
# MDXUSDT
# BTGUSDT
# ARUSDT
# LPTUSDT
# XVGUSDT
# KEEPUSDT
# CFXUSDT
# TKOUSDT


# 2017/12/15~2018/12/30   (黑天鵝，美股熔斷，可用2 hr時間區間）
# start = "15 Dec, 2017"
# test = "17 Dec, 2017"
# end = "30 Aug, 2018"
# 2020/2/15~2020/3/30   (黑天鵝，美股熔斷，可用2 hr時間區間）
# start = "15 Feb, 2020"
# test = "17 Feb, 2020"
# end = "30 Mar, 2020"
# 2020/3/30~2020/12/31 (大牛市，用日線）
# start = "30 Mar, 2020"
# test = "31 Mar, 2020"
# end = "31 Dec, 2020"
# 2020/6/30~2020/8/20  (Defi狂潮，有一些幣大漲，可用8 hr時間區間） 
# start = "28 Jun, 2020"
# test = "30 Jun, 2020"
# end = "22 Aug, 2020"
# 2021/1/1~2021/5/30     (馬斯克頻繁發言、特斯拉，用12 hr時間區間或日線）
# start = "1 Jan, 2021"
# test = "3 Jan, 2021"
# end = "30 May, 2021"
# 2021/5/1~2021/6/19     (大跌，特斯拉消息，用8 hr時間區間）
# start = "1 May, 2021"
# test = "3 May, 2021"
# end = "19 Jun, 2021"


In [195]:
start = "17 Aug, 2017"
end = "31 DEC, 2020"
symbol = "BTCUSDT"

interval = '8h'
# get_fundingRate(symbol,start, end)
# data = write_symbol_dataFrame(symbol, start, end, interval)

# data.to_csv("E:\Data Science\Final Project\DSCode\data\csv\Time_2021_BTC_8h.csv".format(
#     symbol
# ), encoding = 'utf-8', index = False)


#拿csv檔用這行
write_close_toCsv("E:\Data Science\Final Project\DSCode\data\csv\Time_2020_BullMarket_WRX_8h.csv")
#拿dataframe用這個
data = get_close_DataFrame()
print(data)
# data

# changeRate = get_changeRate_DataFrame()
# changeRate
# len(data['BTCUSDT']) == len(changeRate['BTCUSDT'])
# print(changeRate)

                    Date   BTCUSDT  WRXUSDT
0    2020-02-05 08:00:00   4349.99  0.05019
1    2020-02-05 16:00:00   4352.34  0.05020
2    2020-02-06 00:00:00   4285.08  0.06550
3    2020-02-06 08:00:00   4300.25  0.06529
4    2020-02-06 16:00:00   4236.89  0.13066
...                  ...       ...      ...
3689 2020-12-30 00:00:00  27385.00  0.06601
3690 2020-12-30 08:00:00  28386.62  0.06550
3691 2020-12-30 16:00:00  28153.76  0.06558
3692 2020-12-31 00:00:00  28875.54  0.06507
3693 2020-12-31 08:00:00  29155.25  0.06523

[3694 rows x 3 columns]
